In [73]:
!pip install gdown

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [74]:
# natural image
!gdown --id 1-7z0lFddFDcy97On7vO5MlucFUiFJYi1

# adv image
!gdown --id 13ukjGjYmHcxRV94vLccZ6vj_cGyO4CW4

Downloading...
From: https://drive.google.com/uc?id=1-7z0lFddFDcy97On7vO5MlucFUiFJYi1
To: /kaggle/working/lab_img.pt
107MB [00:00, 151MB/s]  
Downloading...
From: https://drive.google.com/uc?id=13ukjGjYmHcxRV94vLccZ6vj_cGyO4CW4
To: /kaggle/working/pgd-resnet101-advs.pt
107MB [00:04, 22.4MB/s] 


In [75]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [76]:
# set device
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print('device', device)

device cuda


In [77]:
# load natural image
nat_data = torch.load('lab_img.pt')

In [78]:
# load adv image
adv_data = torch.load('pgd-resnet101-advs.pt', map_location=torch.device('cpu'))

In [79]:
# load resnet101
model = models.resnet101(pretrained=True)

# use cuda if available
if torch.cuda.is_available():
    model.cuda()

model = model.eval()

print('use cuda', torch.cuda.is_available())

use cuda True


In [80]:
# test two Gaussian Noise ==================================

In [81]:
# define find_sep()
def find_sep(bd, n, n_radius, nat_data):
    seps = []
    for i in range(n):
        nat_stats = []
        for lab, img in nat_data:
            img = img.to(device)
            diff = torch.norm(
                F.softmax(model(img)) - 
                F.softmax(model(img + n_radius * torch.randn_like(img))), 1).item()
            nat_stats.append(diff)
        sep = sorted(nat_stats)[bd]
        seps.append(sep)
    mean_sep = sum(seps) / n
    return mean_sep

In [82]:
# define test TPR
def cal_TPR(
    f_n_radius, 
    s_n_radius, 
    f_mean_sep, 
    s_mean_sep, 
    adv_data, 
    rep):
    
    tprs = []
    
    for i in range(rep):
        
        total = 0
        correct = 0

        for lab, img in adv_data:
            img = img.to(device)
            f_diff = torch.norm(
                F.softmax(model(img)) - 
                F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
            s_diff = torch.norm(
                F.softmax(model(img)) -
                F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
            total += 1
            if f_diff > f_mean_sep and s_diff > s_mean_sep:
                correct += 1
        tprs.append(correct/total)
        
    return sum(tprs)/rep

In [83]:
# define test FPR
def cal_FPR(
    f_n_radius, 
    s_n_radius, 
    f_mean_sep, 
    s_mean_sep, 
    nat_data, 
    rep):
    
    fprs = []
    
    for i in range(rep):
        
        total = 0
        wrong = 0

        for lab, img in nat_data:
            img = img.to(device)
            f_diff = torch.norm(
                F.softmax(model(img)) - 
                F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
            s_diff = torch.norm(
                F.softmax(model(img)) -
                F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
            total += 1
            if f_diff > f_mean_sep and s_diff > s_mean_sep:
                wrong += 1
        fprs.append(wrong/total)
        
    return sum(fprs)/rep

In [ ]:
n_radius = [0.1, 0.2, 0.3, 0.4, 0.5]
result = []
for i in range(5):
    for j in range(i+1, 5):
        f_n_radius = n_radius[i]
        s_n_radius = n_radius[j]
        
        f_mean_sep = find_sep(82, 5, f_n_radius, nat_data)
        s_mean_sep = find_sep(82, 5, s_n_radius, nat_data)
        
        FPR = cal_FPR(f_n_radius, s_n_radius, f_mean_sep, s_mean_sep, nat_data, rep=5)
        TPR = cal_TPR(f_n_radius, s_n_radius, f_mean_sep, s_mean_sep, adv_data, rep=5)
        
        print([f_n_radius, s_n_radius, FPR, TPR])
        result.append([f_n_radius, s_n_radius, FPR, TPR])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as a

[0.1, 0.2, 0.084, 0.8720000000000001]
[0.1, 0.3, 0.074, 0.852]
[0.1, 0.4, 0.042, 0.8560000000000001]
[0.1, 0.5, 0.048, 0.858]
[0.2, 0.3, 0.11399999999999999, 0.9359999999999999]
[0.2, 0.4, 0.07600000000000001, 0.942]
[0.2, 0.5, 0.08, 0.9460000000000001]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
combination = ['radius='+''.join(['[', str(x[0]), '  ', str(x[1]), ']']) for x in result]
TPR = [x[3] for x in result]
plt.barh(combination, TPR)
for index, value in enumerate(TPR):
    plt.text(value, index, f'{int(value*100)}%')
plt.title('relationship between GN and TPR when FPR=0.1')
plt.show()

In [ ]:
# test one Gaussian Noise ==================================

In [ ]:
def find_TPR(mean_sep, rad, adv_data, rep):
    tprs = []
    for i in range(rep):
        total = 0
        correct = 0
        for lab, img in adv_data:
            img = img.to(device)
            diff = torch.norm(
                F.softmax(model(img)) - 
                F.softmax(model(img + rad * torch.randn_like(img))), 1).item()
            total += 1
            if diff > mean_sep:
                correct += 1
        tprs.append(correct/total)
    return sum(tprs)/rep

In [ ]:
def find_FPR(mean_sep, rad, nat_data, rep):
    fprs=[]
    for i in range(rep):
        total = 0
        wrong = 0
        for lab, img in nat_data:
            img = img.to(device)
            diff = torch.norm(
                F.softmax(model(img)) - 
                F.softmax(model(img + rad * torch.randn_like(img))), 1).item()
            total += 1
            if diff > mean_sep:
                wrong += 1
        fprs.append(wrong/total)
    return sum(fprs)/rep

In [ ]:
n_radius = [0.1, 0.2, 0.3, 0.4, 0.5]
ans = []
for rad in n_radius:
    mean_sep = find_sep(91, 5, rad, nat_data)
    tpr = find_TPR(mean_sep, rad, adv_data, rep=5)
    fpr = find_FPR(mean_sep, rad, nat_data, rep=5)
    print(rad, fpr, tpr)
    ans.append([rad, fpr, tpr])
    
    

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
rads = ['radius='+str(x[0]) for x in ans]
tprs = [x[2] for x in ans]
plt.barh(rads, tprs)
for index, value in enumerate(tprs):
    plt.text(value, index, f'{int(value*100)}%')
plt.title('relationship between GN and TPR when FPR=0.1')
plt.show()